In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
import datetime
from matplotlib import pyplot as plt

%matplotlib inline
%run ./data_loading.ipynb

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=0)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=10*10*12, out_features=300)
        self.fc2 = nn.Linear(in_features=300, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=9)
        
    def forward(self, X):
        X = self.relu(self.conv1(X))
        X = self.pool(self.relu(self.conv2(X)))
        X = self.relu(self.conv3(X))
        X = self.pool(self.relu(self.conv4(X)))
        X = X.view(-1, 10*10*12)
        X = self.relu(self.fc1(X))
        X = self.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [3]:
def calculate_accuracy(dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return acc

In [4]:
def train(dataset, epochs, batch_size=5, to_cuda=True, calculate_acc=False):
    train_acc = []
    val_acc = []
    if to_cuda:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in tqdm(enumerate(dataloader, 0)):
            inputs, labels = data
            if to_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = loss_f(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('epoch:', epoch, ', loss:', running_loss * batch_size / dataloader.__len__())
        if calculate_acc:
            t = calculate_accuracy(dataloader)
            v = calculate_accuracy(val_loader)
            print("train acc:", t, "val acc:", v)
            train_acc.append(t)
            val_acc.append(v)
        if epoch % 1 == 0: # every x epochs reshuffle data by creating a new loader
            if to_cuda:
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
            else:
                dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_acc, val_acc

In [5]:
extra_train = Parth_preprocessed_dataset
cr_test = ClassroomDataset(range_10=[7,10])
# Dividing data into 50% train, 40% val, 30% test
scores = []
for i in range(2):
    print("Fold number", i)
    net = Net().float()
    if torch.cuda.is_available():
        net = net.cuda()
    loss_f = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    if i == 0:
        cr_train = ClassroomDataset(range_10=[0,5])
        cr_val = ClassroomDataset(range_10=[5,7])
    else:
        cr_train = ClassroomDataset(range_10=[2,7])
        cr_val = ClassroomDataset(range_10=[0,2])
    acc = []
    for j in range(10):
        val_loader = DataLoader(cr_val, batch_size=1, shuffle=True)
        train(extra_train, epochs=1)
        print(calculate_accuracy(val_loader))
        train_accuracy, val_accuracy = train(cr_train, to_cuda=False, epochs=3, calculate_acc=True)
        print(train_accuracy, val_accuracy)
        acc.append((train_accuracy, val_accuracy))
    scores.append(acc)

Fold number 0


4575it [04:00, 19.04it/s]
11it [00:00, 104.16it/s]

epoch: 0 , loss: 7.902576094714904


640it [00:05, 109.90it/s]


epoch: 0 , loss: 3.168563918932705


...

epoch: 1 , loss: 0.004550078515394018


18it [00:00, 170.98it/s]

train acc: 100.0 val acc: 93.515625


640it [00:03, 165.68it/s]


epoch: 2 , loss: 0.002762366826267293
train acc: 100.0 val acc: 93.515625
[100.0, 100.0, 100.0] [93.28125, 93.515625, 93.515625]


In [6]:
p = "weights/" + str(datetime.datetime.now()).split(".")[0].replace(':','_') + ".pth"
torch.save(net.state_dict(), p)

In [7]:
def class_wise_accuracy(dataloader):
    correct = {}
    total = {}
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            for l in range(len(labels)):
                label = int(labels[l].cpu().numpy())
                pred = predicted[l]
                if label in correct:
                    correct[label] += int(pred == label)
                    total[label] += 1
                else:
                    correct[label] = int(pred == label)
                    total[label] = 1
    acc = {}
    for label in correct:
        acc[label] = 100 * correct[label] / total[label]
    return acc

In [8]:
class_wise_accuracy(val_loader)

{6: 95.0,
 7: 91.875,
 8: 92.3076923076923,
 4: 95.80838323353294,
 3: 90.50632911392405,
 1: 95.65217391304348,
 2: 94.30379746835443,
 5: 92.5}